<a href="https://colab.research.google.com/github/ikram9820/Fastdl_v3-v5/blob/main/dl2%20v5/sd_dreambooth_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq diffusers==0.4.1  transformers gradio ftfy

In [2]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [3]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from tqdm.auto import tqdm

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
path = 'ikram9820/ikram-20-im'
pipe = StableDiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16 ).to("cuda")
path = 'ikram9820/ikram-12-im'
pipe2 = StableDiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16 ).to("cuda")

In [ ]:
prompt = "Portrait of an sks knight with a large moustache, male, detailed face, fantasy, highly detailed, cinematic lighting, digital art painting by greg rutkowski" #@param {type:"string"}
seed =57097832 #@param {type:"number"}
generator =torch.Generator("cuda").manual_seed( seed)
gs = 8 #@param {type:"number"}
step = 100 #@param {type:"number"}

num_samples = 2 #@param {type:"number"}
num_rows = 2 #@param {type:"number"}
all_images = [] 
for _ in range(num_rows):
    images = pipe(prompt, num_images_per_prompt=num_samples,generator=generator, num_inference_steps=step, guidance_scale=gs).images
    all_images.extend(images)

grid = image_grid(all_images, num_samples, num_rows)
grid

In [ ]:
prompt = "Portrait of an sks knight with a large moustache, male, detailed face, fantasy, highly detailed, cinematic lighting, digital art painting by greg rutkowski" #@param {type:"string"}
seed =57097832 #@param {type:"number"}
generator =torch.Generator("cuda").manual_seed( seed)
gs = 8 #@param {type:"number"}
step = 100 #@param {type:"number"}

num_samples = 2 #@param {type:"number"}
num_rows = 2 #@param {type:"number"}
all_images = [] 
for _ in range(num_rows):
    images = pipe2(prompt, num_images_per_prompt=num_samples,generator=generator, num_inference_steps=step, guidance_scale=gs).images
    all_images.extend(images)

grid = image_grid(all_images, num_samples, num_rows)
grid

In [ ]:
import gradio as gr

def inference(prompt, num_samples=1,step:int =50,seed:int=42,gs=7.5):
    step = int(step)
    seed = int(seed)
    all_images = [] 
    images = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=step, guidance_scale=gs).images
    all_images.extend(images)
    return all_images
ex_prompt = "Portrait of an sks knight with a large moustache, male, detailed face, fantasy, highly detailed, cinematic lighting, digital art painting by greg rutkowski"
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="prompt",value = ex_prompt)
            seed = gr.components.Number(label= "seed",value = 57097832)
            gs = gr.components.Number(label="guidance_scale",value=7.5)
            step = gr.components.Number(label= "step",value = 50)
            samples = gr.Slider(label="Samples",value=1)
            run = gr.Button(value="Run")
        with gr.Column():
            gallery = gr.Gallery(show_label=False)

    run.click(inference, inputs=[prompt,samples,step,seed,gs], outputs=gallery)
    gr.Examples([[ex_prompt, 1,1]], [prompt,samples], gallery, inference, cache_examples=False)


demo.launch(debug=True)
